# Time Value of Money

In [1]:
import numpy_financial as npf
import pandas as pd
import numpy as np

In [2]:
[fn for fn in dir(npf) if not fn.startswith("_")]

['fv', 'ipmt', 'irr', 'mirr', 'nper', 'npv', 'pmt', 'ppmt', 'pv', 'rate']

In [3]:
npf.__version__

'1.0.0'

What is the future value after 10 years with monthly savings of $250? Assume the interest rate offered by the bank is 6% compounded monthly.

FVIFA = P * ([(1 + r)^n] - 1) / n

r = 0.06 / 12 p.a

n = 10 * 12 years

P = $250

In [4]:
# params: rate, nper, pmt, pv, when="end"
npf.fv(0.06/12, 10*12, -250, 0)

40969.836701614506

What is the future value after 15 years of saving $1000 now, with an additional monthly savings of $150? Assume the interest rate offered by the bank is 3% compounded monthly.

In [5]:
npf.fv(0.03/12, 15*12, -150, -1000)

35613.33520474759

## FI Question:

A 7-year, 10% annual pay bond has a par value of Rs. 1000. What would this be trading for if it were being priced to yield 13% at an annual rate?

In [6]:
npf.pv(0.13, 7, -100, -1000)

867.3216870183404

## Suppose you take a home loan:

* Loan Amount: $360000
* Interest Rate: 5.875%
* Years to pay back: 30 years

Calculate:
1. How much is the monthly payment?
2. How sensitive is the payment to interest rate?

In [7]:
help(npf.pmt)

Help on function pmt in module numpy_financial._financial:

pmt(rate, nper, pv, fv=0, when='end')
    Compute the payment against loan principal plus interest.
    
    Given:
     * a present value, `pv` (e.g., an amount borrowed)
     * a future value, `fv` (e.g., 0)
     * an interest `rate` compounded once per period, of which
       there are
     * `nper` total
     * and (optional) specification of whether payment is made
       at the beginning (`when` = {'begin', 1}) or the end
       (`when` = {'end', 0}) of each period
    
    Return:
       the (fixed) periodic payment.
    
    Parameters
    ----------
    rate : array_like
        Rate of interest (per period)
    nper : array_like
        Number of compounding periods
    pv : array_like
        Present value
    fv : array_like,  optional
        Future value (default = 0)
    when : {{'begin', 1}, {'end', 0}}, {string, int}
        When payments are due ('begin' (1) or 'end' (0))
    
    Returns
    -------
    out 

In [8]:
rate = 5.875 # in %
loan_amt = 360000
term = 30

In [9]:
mon_pmt = npf.pmt(rate/1200, term*12, -loan_amt)
print(f"We have to pay ${mon_pmt:,.2f} per month.")

We have to pay $2,129.54 per month.


In [10]:
rates = [(rate -0.5 + x/4) / 1200 for x in range(5)]
rates

[0.004479166666666667,
 0.0046875,
 0.004895833333333334,
 0.005104166666666667,
 0.0053125]

In [11]:
pmts = npf.pmt(rates, term*12, -loan_amt)

payments_data = {"Interest": [], "Monthly_Pay": []}

for i, pay in enumerate(pmts):
  payments_data["Interest"].append(rates[i] * 1200)
  payments_data["Monthly_Pay"].append(round(pay, 2))

pd.DataFrame(payments_data)

,Interest,Monthly_Pay
0,5.375,2015.90
1,5.625,2072.36
2,5.875,2129.54
3,6.125,2187.40
4,6.375,2245.93


## Suppose you receive for the next few years
* Annuity amount: $1000
* Expected Return: 8%
* Life: 5 years

1. How much is $1000 received a year from now worth today?
2. What is the value of the entire annuity in today's dollar?
3. How much is each installment worth over the five years?

In [12]:
help(npf.pv)

Help on function pv in module numpy_financial._financial:

pv(rate, nper, pmt, fv=0, when='end')
    Compute the present value.
    
    Given:
     * a future value, `fv`
     * an interest `rate` compounded once per period, of which
       there are
     * `nper` total
     * a (fixed) payment, `pmt`, paid either
     * at the beginning (`when` = {'begin', 1}) or the end
       (`when` = {'end', 0}) of each period
    
    Return:
       the value now
    
    Parameters
    ----------
    rate : array_like
        Rate of interest (per period)
    nper : array_like
        Number of compounding periods
    pmt : array_like
        Payment
    fv : array_like, optional
        Future value
    when : {{'begin', 1}, {'end', 0}}, {string, int}, optional
        When payments are due ('begin' (1) or 'end' (0))
    
    Returns
    -------
    out : ndarray, float
        Present value of a series of payments or investments.
    
    Notes
    -----
    The present value is computed by s

In [13]:
cashflow = 1000
r = 0.08
T = 5

### Cashflow at the end of first year.

In [14]:
npf.pv(r, 1, 0, -cashflow)

925.9259259259259

### Cashflow of last year

In [15]:
npf.pv(r, T, 0, -cashflow)

680.5831970337529

### Present value of the total investment

In [16]:
npf.pv(r, T, -cashflow)

3992.7100370780886

### Cashflow for each year

In [17]:
T = range(1, 6)

cashflows = npf.pv(r, T, 0, -cashflow)
cashflows

array([925.92592593, 857.3388203 , 793.83224102, 735.0298528 ,
       680.58319703])

In [18]:
print(f"\n{'Year':10}{'PV of Cash Flow'}")
print(f"-" * 25)

for year in T:
  pv_cashflow = f"$ {cashflows[year - 1]:,.2f}"
  print(f"{year}{pv_cashflow:>24}")


Year      PV of Cash Flow
-------------------------
1                $ 925.93
2                $ 857.34
3                $ 793.83
4                $ 735.03
5                $ 680.58


### Cumsum of each year

In [19]:
np.cumsum(cashflows).round(2)

array([ 925.93, 1783.26, 2577.1 , 3312.13, 3992.71])

## NPV & IRR Funtions

### Suppose you have a capital project:
* n = 5 years
* initital investment = $425,000
* Cost of capital = 12%
* Net annual cashflow = $150,000

1. What is the Net Present Value of the project?
2. What is the internal rate of return?

In [20]:
help(npf.npv)

Help on function npv in module numpy_financial._financial:

npv(rate, values)
    Returns the NPV (Net Present Value) of a cash flow series.
    
    Parameters
    ----------
    rate : scalar
        The discount rate.
    values : array_like, shape(M, )
        The values of the time series of cash flows.  The (fixed) time
        interval between cash flow "events" must be the same as that for
        which `rate` is given (i.e., if `rate` is per year, then precisely
        a year is understood to elapse between each cash flow event).  By
        convention, investments or "deposits" are negative, income or
        "withdrawals" are positive; `values` must begin with the initial
        investment, thus `values[0]` will typically be negative.
    
    Returns
    -------
    out : float
        The NPV of the input cash flow series `values` at the discount
        `rate`.
    
    Warnings
    --------
    ``npv`` considers a series of cashflows starting in the present (t = 0).
  

In [21]:
kc = 0.12
cash_inflows = [-425000]
n_years = 5

i = 0
while i < n_years:
  cash_inflows.append(150000)
  i+=1

print(cash_inflows)

[-425000, 150000, 150000, 150000, 150000, 150000]


In [22]:
npv_project = npf.npv(kc, cash_inflows)
print(f"NPV = ${npv_project:,.2f} | We accept the project!")

NPV = $115,716.43 | We accept the project!


In [23]:
# More verbose way
discounted = []
for n, cashflow in enumerate(cash_inflows):
  discounted.append(cashflow / (1 + kc)**n)

print(sum(discounted) == npv_project)

True


In [24]:
help(npf.irr)

Help on function irr in module numpy_financial._financial:

irr(values)
    Return the Internal Rate of Return (IRR).
    
    This is the "average" periodically compounded rate of return
    that gives a net present value of 0.0; for a more complete explanation,
    see Notes below.
    
    :class:`decimal.Decimal` type is not supported.
    
    Parameters
    ----------
    values : array_like, shape(N,)
        Input cash flows per time period.  By convention, net "deposits"
        are negative and net "withdrawals" are positive.  Thus, for
        example, at least the first element of `values`, which represents
        the initial investment, will typically be negative.
    
    Returns
    -------
    out : float
        Internal Rate of Return for periodic input values.
    
    Notes
    -----
    The IRR is perhaps best understood through an example (illustrated
    using np.irr in the Examples section below).  Suppose one invests 100
    units and then makes the following 

In [25]:
irr = npf.irr(cash_inflows)
print(f"IRR = {irr*100:.2f}%")

IRR = 22.50%


In [26]:
npf.npv(irr, cash_inflows) # Almost zero

3.637978807091713e-10

In [29]:
npf.fv(0.06/12, 10*12, -100, 0)

16387.9346806458